In [61]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
import chromadb
import os
from llama_index.readers.file import PDFReader
from llama_index.vector_stores.chroma import ChromaVectorStore


def load_pdf_for_rag(pdf_path, storage_context):
    # Load the pdf as documents
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"The PDF file '{pdf_path}' does not exist.")

    pdf_reader = PDFReader()
    documents = pdf_reader.load_data(file=pdf_path)

    # Initialize the VectorStoreIndex with the loaded documents
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    return index

def load_index_for_rag(index_name):
    # generate database name to load
    db_name = "index_dbs/" + index_name + "_db"
    if not os.path.exists(db_name):
        create_db = True
    else:
        create_db = False

    # initialize client, setting path to save data
    db = chromadb.PersistentClient(path=db_name)

    # create collection
    chroma_collection = db.get_or_create_collection("quickstart")

    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    if create_db:
        # load index from pdf file and create the database for next time
        pdf_path = "data/" + index_name + ".pdf"
        index = load_pdf_for_rag(pdf_path, storage_context)
    else:
        # load index from database
        index = VectorStoreIndex.from_vector_store(
            vector_store, storage_context=storage_context
        )

    # Create a retriever from the index
    #retriever = index.as_retriever()
    retriever = index.as_retriever(retrieval_mode='similarity', k=4)
    return retriever

# The score_threshold determines how relevant a node's text must be to add it
#  to the content.  Typically 0.6 or higher yields relevant content.  Default
#  is 0, which means any returned node will be used.
def fetch_relevant_documents(message, retriever, score_threshold=0.0):
    if retriever is None:
        raise ValueError("Retriever has not been initialized. Call load_pdf_for_rag() first.")
    
    # Retrieve relevant nodes based on the input message
    retrieved_nodes = retriever.retrieve(message)
    print(f"{len(retrieved_nodes)} documents found.")

    if not retrieved_nodes:
        return "No relevant information found."
    
    # Combine the content of retrieved nodes into a single string
    relevant_content = "\n\n\n"
    for node in retrieved_nodes:
        print(f"node = {node}\n\n")
        if node.score > score_threshold:
            # only add content that is above the score threshold
            relevant_content += node.node.text
            print(f"relevant_content = {relevant_content}\n\n")
    return relevant_content

In [62]:
import os
from dotenv import load_dotenv
import chainlit as cl
import openai
import asyncio
import json
from datetime import datetime
from prompts import ASSESSMENT_PROMPT, SYSTEM_PROMPT, CLASS_CONTEXT, CHARACTER_CREATION
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from player_record import read_player_record, write_player_record, format_player_record, parse_player_record

# Load environment variables
load_dotenv()
retriever = load_index_for_rag("PlayerDnDBasicRules_v0.2_PrintFriendly")


In [46]:
#fetch_relevant_documents("asdfasdfadsf", retriever)

query = "ranger class"

# Retrieve relevant documents
relevant_docs = retriever.retrieve(query)

print(f"Number of relevant documents: {len(relevant_docs)}")
print("\n" + "="*50 + "\n")

for i, doc in enumerate(relevant_docs):
    print(f"Document {i+1}:")
    print(f"Text sample: {doc.node.get_content()}...")  # Print first 200 characters
    print(f"Metadata: {doc.node.metadata}")
    print(f"Score: {doc.score}")
    print("\n" + "="*50 + "\n")

2024-09-29 17:24:26 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Number of relevant documents: 2


Document 1:
Text sample: All of these heroes are fighters, perhaps the most 
diverse class of characters in the worlds of Dungeons & 
Dragons . Questing knights, conquering overlords, royal 
champions, elite foot soldiers, hardened mercenaries, 
and bandit kings—as fighters, they all share an 
unparalleled mastery with weapons and armor, and a 
thorough knowledge of the skills of combat. And they 
are well acquainted with death, both meting it out and 
staring it defiantly in the face.
Well-Rounded Specialists
Fighters learn the basics of all combat styles. Every 
fighter can swing an axe, fence with a rapier, wield a 
longsword or a greatsword, use a bow, and even trap foes 
in a net with some degree of skill. Likewise, a fighter is 
adept with shields and every form of armor. Beyond that 
basic degree of familiarity, each fighter specializes in a 
certain 

In [66]:
fetch_relevant_documents("I want to kill a creature", retriever, 0.6)

2024-09-29 17:42:53 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2 documents found.
node = Node ID: e8109518-a53b-4e40-b398-0243aedc6a31
Text: At Higher Levels.  When you cast this spell using  a spell slot
of 2nd level or higher, you can target one  additional creature for
each slot level above 1st. The  creatures must be within 30 feet of
each other when you  target them. Command 1st-level enchantment
Casting Time: 1 action Range: 60 feet Components:  V Duration:  1
round You speak a...
Score:  0.673



relevant_content = 


At Higher Levels.  When you cast this spell using 
a spell slot of 2nd level or higher, you can target one 
additional creature for each slot level above 1st. The 
creatures must be within 30 feet of each other when you 
target them.
Command
1st-level enchantment
Casting Time: 1 action
Range: 60 feet
Components:  V
Duration:  1 round
You speak a one-word command to a creature you can 
see within range. The target must succeed on a W

'\n\n\nAt Higher Levels.  When you cast this spell using \na spell slot of 2nd level or higher, you can target one \nadditional creature for each slot level above 1st. The \ncreatures must be within 30 feet of each other when you \ntarget them.\nCommand\n1st-level enchantment\nCasting Time: 1 action\nRange: 60 feet\nComponents:  V\nDuration:  1 round\nYou speak a one-word command to a creature you can \nsee within range. The target must succeed on a Wisdom \nsaving throw or follow the command on its next turn. \nThe spell has no effect if the target is undead, if it \ndoesn’t understand your language, or if your command \nis directly harmful to it.\n Some typical commands and their effects follow. You \nmight issue a command other than one described here. \nIf you do so, the DM determines how the target behaves. \nIf the target can’t follow your command, the spell ends.\n Approach.  The target moves toward you by the \nshortest and most direct route, ending its turn if it \nmoves withi